In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("Measurement_summary.csv")

In [12]:
df.head(3)

,Measurement date,Station code,Address,Latitude,Longitude,SO2,NO2,O3,CO,PM10,PM2.5
0,2017-01-01 00:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0
1,2017-01-01 01:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.058,0.002,1.2,71.0,59.0
2,2017-01-01 02:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,59.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647511 entries, 0 to 647510
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Measurement date  647511 non-null  object 
 1   Station code      647511 non-null  int64  
 2   Address           647511 non-null  object 
 3   Latitude          647511 non-null  float64
 4   Longitude         647511 non-null  float64
 5   SO2               647511 non-null  float64
 6   NO2               647511 non-null  float64
 7   O3                647511 non-null  float64
 8   CO                647511 non-null  float64
 9   PM10              647511 non-null  float64
 10  PM2.5             647511 non-null  float64
dtypes: float64(8), int64(1), object(2)
memory usage: 54.3+ MB


In [7]:
# mengganti dtype measurement date menjadi datetime
df['Measurement date'] = pd.to_datetime(df['Measurement date'])
df['Measurement_date'] = df['Measurement date'].dt.date
df['Measurement_time'] = df['Measurement date'].dt.time

In [8]:
df = df.rename(columns={'Station code': 'Station_code'})

In [10]:
# function untuk membuat kolom district
def extract_district(address):
    try:
        parts = address.split(', ')
        district = parts[2]
        return district
    except:
        return None

df['District'] = df['Address'].apply(extract_district)
new_cols = ['Measurement_date', 'Measurement_time', 'Station_code', 'Address', 'District', 'Latitude', 'Longitude', 'SO2', 'NO2', 'O3', 'CO', 'PM10', 'PM2.5']
df = df[new_cols]
df['Measurement_date'] = pd.to_datetime(df['Measurement_date'])

# function untuk membuat kolom air_quality index berdasarkan PM2.5
def calculate_aqi_pm25(pm25):
    if pm25 <= 30:
        return round((50 / 30) * pm25, 1)
    elif pm25 <= 60:
        return round(((100 - 51) / (60 - 31)) * (pm25 - 31) + 51, 1)
    elif pm25 <= 90:
        return round(((200 - 101) / (90 - 61)) * (pm25 - 61) + 101, 1)
    elif pm25 <= 120:
        return round(((300 - 201) / (120 - 91)) * (pm25 - 91) + 201, 1)
    elif pm25 <= 250:
        return round(((400 - 301) / (250 - 121)) * (pm25 - 121) + 301, 1)
    else:
        return round(((500 - 401) / (350 - 251)) * (pm25 - 251) + 401, 1)

# fuction untuk menentukan kategori berdasarkan AQI_index
def categorize_aqi(aqi):
    if aqi <= 50:
        return "Good"
    elif aqi <= 100:
        return "Satisfactory"
    elif aqi <= 200:
        return "Moderate"
    elif aqi <= 300:
        return "Poor"
    elif aqi <= 400:
        return "Very Poor"
    else:
        return "Severe"

df['AQI_index'] = df['PM2.5'].apply(calculate_aqi_pm25)
df['AQI_category'] = df['AQI_index'].apply(categorize_aqi) 
df.to_csv('measurement_datamart.csv', index=False)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_1528\2442991077.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Measurement_date'] = pd.to_datetime(df['Measurement_date'])


In [11]:
df.head(3)

,Measurement_date,Measurement_time,Station_code,Address,District,Latitude,Longitude,SO2,NO2,O3,CO,PM10,PM2.5,AQI_index,AQI_category
0,2017-01-01,00:00:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",Jongno-gu,37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0,94.9,Satisfactory
1,2017-01-01,01:00:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",Jongno-gu,37.572016,127.005008,0.004,0.058,0.002,1.2,71.0,59.0,98.3,Satisfactory
2,2017-01-01,02:00:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",Jongno-gu,37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,59.0,98.3,Satisfactory
